In [15]:
import pandas as pd
# data = pd.read_csv('D:/ShipSupplies/DA/data/raw.csv')
df = pd.read_csv('../data/raw_postpro_test.csv')
df = df[(df['출고운반선'] != 0) & (df['출고운반선' ] != '0')]

In [16]:
from gensim.models import FastText
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 제일 좋은 모델
X = list(df["Subject"]+ ' ' +df['출고운반선']+  ' ' + df['key2'] + ' ' + df['Assembly'] + ' '  + df['견적화폐']  +  ' ' + df['발주처'])

# r2 74
# X = list(df["청구품목"]+ ' '  +df['출고운반선']+  ' ' + df['key2'] + ' ' + df['Assembly'] + ' '  + df['견적화폐']  +  ' ' + df['발주처'])

# X = list(df["청구품목"]+ ' '  + df['key2'] + ' ' + df['Assembly'] + ' '  + df['견적화폐']  +  ' ' + df['발주처'] +  ' ' + df['Control No.'])

from sklearn.preprocessing import StandardScaler
# target 값
y = df['leadtime']

from tensorflow.keras.preprocessing.text import Tokenizer
# 텍스트 데이터를 정수 시퀀스로 변환
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

In [17]:
len(sequences)
# df

18896

In [18]:
X[:10]

['NO.1 A/E C18 MAJOR O/H PARTS 본선 선적 COOLER 323-6480 LINES GP-FUEL  KRW HAEIN Coporation_Cheonan',
 '운전시간 대비 #2 AUX ENG TOP END O/H PARTS 본선 선적 COOLER GASKET KIT KRW HAEIN Coporation_Cheonan',
 '운전시간 대비 #2 AUX ENG(CAT 3512C) MAJOR O/H PARTS SANWA FONTAINE COOLER 285-8374 MANIFOLD GP-EXH  KRW HAEIN Coporation_Cheonan',
 '#1 AUX ENG TOP END O/H PARTS 체항수리 시 사용 COOLER 159-8828 TURBO GP KRW HAEIN Coporation_Cheonan',
 'CAR-D-ENG-210603-01 R/C OCEANUS COOLER 8N-6224 PISTON GP-ROD& KRW HAEIN Coporation_Cheonan',
 '운전시간 대비 #3 AUX ENG(CAT 3508B) MAJOR O/H PARTS 발주의 건 본선 선적 COOLER 8N-6151 PUMP GP-F TFR  KRW HAEIN Coporation_Cheonan',
 'OCA #2 AUX ENG MAJOR O/H PART OCA 사용 COOLER 8N-7174 GEAR GP-FRONT  KRW HAEIN Coporation_Cheonan',
 '운전 시간 대비 3508B MAJOR O/H PARTS R/C HANARO COOLER 8N-7174 GEAR GP-FRONT  KRW HAEIN Coporation_Cheonan',
 '#3 AUX ENG (CAT 3508B) MAJOR OVERHAUL PARTS R/C VICTORIA-A COOLER 8N-7174 GEAR GP-FRONT  KRW HAEIN Coporation_Cheonan',
 '3512B GEAR TRAIN PARTS HAND-CARRY COOL

In [19]:
# 시퀀스 길이를 정하기 위해 X 문자열 길이 확인
lengths = [len(text) for text in X]
average_length = sum(lengths) / len(lengths)
max_length = max(lengths)

print("Average length:", average_length)
print("Max length:", max_length)

Average length: 93.95602243861134
Max length: 181


In [20]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM, Conv1D, MaxPooling1D, Dense, Flatten
from tensorflow.keras.preprocessing import sequence

from tensorflow.keras.preprocessing.sequence import pad_sequences

# 시퀀스 패딩
max_sequence_length = 100  # 적절한 시퀀스 길이로 설정
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)

# y_train과 y_test 정규화
scaler = StandardScaler()
y_train = scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test = scaler.transform(y_test.values.reshape(-1, 1))

# 모델 구성
model = Sequential()
model.add(Embedding(input_dim=12309, output_dim=100, input_length=max_sequence_length))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(64))
model.add(Dense(1))
 
# 모델 컴파일
model.compile(loss='mean_squared_error', optimizer='adam')

In [21]:
X_train.shape, y_train.shape
# len(X), len(y)

((15116, 100), (15116, 1))

In [22]:
X_test.shape, y_test.shape

((3780, 100), (3780, 1))

In [23]:
X_train

array([[  0,   0,   0, ...,  29,  30,  31],
       [  0,   0,   0, ...,  37,  26,  23],
       [  0,   0,   0, ...,  21,  43,  88],
       ...,
       [  0,   0,   0, ...,  21,  43,  88],
       [  0,   0,   0, ..., 159,  37, 160],
       [  0,   0,   0, ..., 150,  26,  23]])

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          1230900   
                                                                 
 conv1d_1 (Conv1D)           (None, 98, 64)            19264     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 49, 64)           0         
 1D)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,283,253
Trainable params: 1,283,253
Non-trainable params: 0
____________________________________________

In [25]:
import matplotlib.pyplot as plt

# 모델 훈련
history = model.fit(X_train, y_train, epochs=14, batch_size=128, validation_data=(X_test, y_test))

# 손실 값 추출
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# 에포크 수
epochs = range(1, len(train_loss) + 1)

# 학습 곡선 그리기
plt.plot(epochs, train_loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Epoch 1/14


UnimplementedError: Graph execution error:

Detected at node 'sequential_1/conv1d_1/Conv1D' defined at (most recent call last):
    File "C:\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "D:\ShipSupplies\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "D:\ShipSupplies\venv\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "D:\ShipSupplies\venv\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "D:\ShipSupplies\venv\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Python310\lib\asyncio\base_events.py", line 603, in run_forever
      self._run_once()
    File "C:\Python310\lib\asyncio\base_events.py

In [68]:
y_pred = model.predict(X_test)

# 예측 결과 출력
print("Predicted values:")
print(y_pred)

print("Actual values:")
print(y_test)

92/92 [==============================] - 1s 13ms/step
Predicted values:
[[-0.15532874]
 [-0.28113848]
 [ 0.47491866]
 ...
 [-0.9085509 ]
 [-0.9076042 ]
 [ 0.00826609]]
Actual values:
[[-0.77469142]
 [-0.83039049]
 [ 1.45327105]
 ...
 [-0.77469142]
 [-0.88608955]
 [ 0.25574122]]


In [69]:
# 원래 스케일로 되돌리기
y_test = scaler.inverse_transform(y_test)
y_pred = scaler.inverse_transform(y_pred)

import numpy as np
y_test_arr = np.array(y_test)
y_pred_arr = np.array(y_pred)

# 값들을 비교하면서 출력합니다.
for i in range(40):
    print("Actual:", y_test_arr[i], "Predicted:", y_pred_arr[i])

Actual: [22.] Predicted: [66.47922]
Actual: [18.] Predicted: [57.444256]
Actual: [182.] Predicted: [111.74012]
Actual: [65.] Predicted: [54.590435]
Actual: [40.] Predicted: [79.697235]
Actual: [146.] Predicted: [98.79101]
Actual: [230.] Predicted: [230.5834]
Actual: [10.] Predicted: [49.27442]
Actual: [59.] Predicted: [136.67262]
Actual: [1.] Predicted: [54.711903]
Actual: [370.] Predicted: [359.5622]
Actual: [19.] Predicted: [13.493227]
Actual: [38.] Predicted: [132.34088]
Actual: [4.] Predicted: [9.280672]
Actual: [78.] Predicted: [106.952415]
Actual: [91.] Predicted: [127.86617]
Actual: [44.] Predicted: [67.84815]
Actual: [10.] Predicted: [38.581512]
Actual: [53.] Predicted: [43.097218]
Actual: [99.] Predicted: [97.58123]
Actual: [43.] Predicted: [122.2325]
Actual: [18.] Predicted: [38.127316]
Actual: [3.] Predicted: [109.12754]
Actual: [112.] Predicted: [15.387011]
Actual: [80.] Predicted: [97.73331]
Actual: [22.] Predicted: [17.83954]
Actual: [267.] Predicted: [39.265953]
Actual: 

In [70]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

R^2 Score: 0.48918194868322895
